In [38]:
import pandas as pd
from tqdm import tqdm_notebook, tqdm
from collections import Counter

uid = pd.read_csv('uid_click_log.csv')

In [39]:
uid.head()

,uid,vid,cid,playtime,duration,date,rank
0,100000,100870,100037,4966,9590,20200706,1
1,100000,101167,100024,5037,4949,20200706,2
2,100000,103608,100008,5137,14884,20200706,3
3,100000,100220,100084,5139,16784,20200706,4
4,100000,101674,100027,5149,5760,20200706,5


In [40]:
example = pd.read_csv('example.csv')
example

,uid,vid
0,100000,1
1,100000,2
2,100000,3
3,100000,4
4,100000,5
...,...,...
25190,105998,1
25191,105998,2
25192,105998,3
25193,105998,4


In [42]:
from collections import defaultdict

def merge_dicts_by_adding_values(*dicts):
    merged_dict = defaultdict(int)
    for d in dicts:
        for key, value in d.items():
            merged_dict[key] += value
    return dict(merged_dict)

{'a': 4, 'b': 5, 'c': 4, 'd': 5}


In [43]:
uid['uid'].nunique(), uid['vid'].nunique()

(6000, 9992)

In [44]:
val_data = uid.groupby('uid').tail(5)
train_data = uid[~uid.index.isin(val_data.index)]

In [45]:
freq = {}
for u, df in tqdm(train_data.groupby('uid')):
    for v1, v2 in zip(df['vid'].values[:-1], df['vid'].values[1:]):
        if v1 not in freq:
            freq[v1] = {v2:1}
        else:
            if v2 in freq[v1]:
                freq[v1][v2] += 1
            else:
                freq[v1][v2] = 1

100%|██████████| 6000/6000 [00:02<00:00, 2254.47it/s]


In [46]:
recall_top5 = 0
for u, df in tqdm(train_data.iloc[:].groupby('uid')):
    df10 = df.tail(3)
    pred_freq = merge_dicts_by_adding_values(*[freq[x] for x in df10['vid'].values if x in freq])
    pred_vids = [k for k, v in sorted(pred_freq.items(), key=lambda item: item[1])][::-1]
    pred_vids_top5 = [x for x in pred_vids if x not in df['vid'].values][:5]
    recall_top5 += int(len(set(pred_vids_top5) & set(val_data[val_data['uid'] == u]['vid'].values)) > 0)

100%|██████████| 6000/6000 [00:36<00:00, 165.99it/s]


In [47]:
recall_top5

619

In [53]:
submit = []
for u, df in tqdm(uid.groupby('uid')):
    if u not in example['uid'].values:
        continue

    df10 = df.tail(3)
    pred_freq = merge_dicts_by_adding_values(*[freq[x] for x in df10['vid'].values if x in freq])
    pred_vids = [k for k, v in sorted(pred_freq.items(), key=lambda item: item[1])][::-1]
    pred_vids = pred_vids[:5]
    # pred_vids_top5 = [x for x in pred_vids if x not in df['vid'].values][:5]

    # if len(pred_vids_top5) <= 5:
    #     pred_vids_top5 += [102477] * (5-len(pred_vids_top5))
    for v in pred_vids_top5[:5]:
        submit.append([u, v])

100%|██████████| 6000/6000 [00:02<00:00, 2022.89it/s]


In [54]:
submit = pd.DataFrame(submit)
submit.columns = ['uid', 'vid']

In [59]:
submit.to_csv('s2.csv', index=None)